In [ ]:
import sagemaker
import boto3
from sagemaker import image_uris
import numpy as np

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

# get the URI for the XGBoost container
container_image = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

# build a SageMaker estimator class
xgb_estimator = sagemaker.estimator.Estimator(
    container_image,
    role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/iris/output'.format(bucket),
    sagemaker_session=sagemaker_session
)

# set the hyperparameters
xgb_estimator.set_hyperparameters(
                        num_class=2,
                        silent=0,
                        objective='multi:softmax',
                        num_round=10
)

#### Uploading the Training Dataset

In [ ]:
# Upload the dataset to our S3 bucket
input_train = sagemaker_session.upload_data(path='train.csv', key_prefix='titanic')
input_val = sagemaker_session.upload_data(path='val.csv', key_prefix='titanic')

#### Start Training

In [ ]:
# Now run training against the training and val sets created above
# Refer to the SageMaker training console
xgb_estimator.fit({
    'train': input_train,
    'validation': input_val
})